In [1]:
import pandas as pd
import numpy as np

# restaurants_limited_features
df_res = pd.read_csv("/Users/dima/code/Dimasaur/scorecast/data/restaurants_limited_features.csv")

# restaurant ids dataset
res_ids = pd.read_csv("/Users/dima/code/Dimasaur/scorecast/data/restaurants.csv")

# yelp general business dataset
business_json_path = "/Users/dima/code/Dimasaur/scorecast/data/yelp_academic_dataset_business.json"

#reading the json file
df_business_raw = pd.read_json(business_json_path, lines=True)

# creating a destination path
csv_file_path_review = "yelp_business_raw.csv"

#converting the output into a csv file
df_business_raw.to_csv(csv_file_path_review, index=False)



df_res['food_type_string'] = df_res['food_type'].apply(lambda x: str(x).strip('[]'))
df_res['food_type_string'] = df_res['food_type_string'].apply(lambda x: str(x).replace("'",""))

# replace the values in the food_types with the top - 20 cuisines

replace_dictionary = {
    "Pizza" : "Italian",
    "Burgers, Fast Food" : "Burgers",
    "Italian, Pizza" : "Italian",
    "American (New)" : "American",
    "Fast Food, Sandwiches" : "Sandwiches",
    "American (Traditional)" : "American",
    "Fast Food, Mexican" : "Mexican",
    "Coffee & Tea, Food" : "Cafes",
    "Japanese, Sushi Bars" : "Japanese",
    "American (Traditional), Diners" : "Diners",
    "Bakeries, Food" : "Bakery",
    "Delis, Sandwiches" : "Delis",
    "Mexican, Tex-Mex" : "Mexican",
    "American (Traditional), Burgers" : "American",
    "Sushi Bars" : "Japanese",
    "Chinese, Fast Food" : "Chinese",
    "American (Traditional), Cafes" : "American",
    "Mexican, Tacos" : "Mexican",
    "Pizza, Sandwiches" : "Sandwiches",
    "Food, Pizza" : "Italian",
    "American (Traditional), Seafood" : "American",
    "Greek, Mediterranean" : "Greek",
    "American (New), American (Traditional)" : "American",
    "Cajun/Creole, Seafood" : "Cajun/Creole"

}

df_res.food_type_string = df_res.food_type_string.replace(replace_dictionary)

# generate the clea array of the top 20 categories
df_food_t20 = list(df_res.food_type_string.value_counts(normalize=True).head(25).index)
df_food_t20.remove('O')
df_food_t20.remove('')
df_food_t20 = df_food_t20[:20]

# top 20 food types
np_food_20 = np.array([df_food_t20])


# filtering the the list of business to get the restaurants only
df_restaurants = df_business_raw[df_business_raw.business_id.isin(res_ids.business_id)].reset_index(drop=True)


df_restaurants.categories_clean = df_restaurants.categories.str.replace(", ",",")

df_restaurants['food_type_one'] = None

# function that extracts a category from a column in a dataframe if that category is present in a given list


def extract_category(categories, food_types):

    # Combine the lists after splitting by space and by comma
    possible_foods = categories.split(' ') + categories.split(',')
    # Remove any empty strings that might be created by split
    possible_foods = [food.strip() for food in possible_foods if food.strip()]

    for food in possible_foods:
        if food in food_types:
            return food
    return None

df_restaurants['food_type_one'] = df_restaurants['categories'].apply(lambda x: extract_category(x, df_food_t20))


/var/folders/fd/585jf60x39vflmpy4gfz7yzh0000gn/T/ipykernel_78090/1367318593.py:5: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_res = pd.read_csv("/Users/dima/code/Dimasaur/scorecast/data/restaurants_limited_features.csv")
/var/folders/fd/585jf60x39vflmpy4gfz7yzh0000gn/T/ipykernel_78090/1367318593.py:73: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_restaurants.categories_clean = df_restaurants.categories.str.replace(", ",",")


In [3]:
df_restaurants.groupby(['state','food_type_one']).stars.mean().sort_values(ascending=False).tail(50)

state  food_type_one
AB     Mexican          3.361702
LA     Sandwiches       3.349359
IN     Sandwiches       3.342752
PA     Diners           3.340541
MO     Sandwiches       3.328502
DE     American         3.321429
FL     Chinese          3.320175
ID     Mexican          3.316000
NJ     Diners           3.297297
IL     Mexican          3.291209
AZ     Chinese          3.280576
TN     Italian          3.275000
MO     Mexican          3.258278
AB     Steakhouses      3.244186
IL     Italian          3.241667
AB     American         3.226316
TN     Chinese          3.223333
CA     Chicken Wings    3.187500
IL     Greek            3.166667
NV     Chinese          3.155844
DE     Chicken Wings    3.152174
       Chinese          3.149123
NV     Chicken Wings    3.144737
ID     Chinese          3.125000
AZ     Cajun/Creole     3.090909
IL     Sandwiches       3.086420
LA     Chicken Wings    3.042683
NJ     Chicken Wings    3.033333
AB     Burgers          3.019774
TN     Chicken Wings  

In [4]:
df_xms = df_restaurants[df_restaurants.state == 'XMS']

df_xms

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,food_type_one
34288,IvQs2hhXKZ4NgZhVZ5jjjQ,Pumpernickles Catering,95a Mount Pleasant,Liverpool,XMS,L3 5TB,38.673049,-90.423156,4.0,5,1,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","Caterers, Breakfast & Brunch, Convenience Stor...",None,None


In [5]:
unique_states = pd.DataFrame(df_restaurants['state'].unique(), columns=['state'])
unique_States = unique_states.state.str.upper()
unique_states = pd.DataFrame(unique_states.state.sort_values(ascending=True).reset_index(drop=True))

unique_cities = pd.DataFrame(df_restaurants['city'].unique(), columns=['city'])
unique_cities = pd.DataFrame(unique_cities.city.str.upper())
unique_cities = pd.DataFrame(unique_cities.city.sort_values(ascending=True).reset_index(drop=True))


In [6]:
unique_cities.to_csv('cities.csv')
unique_states.to_csv('states.csv')

In [21]:
df_restaurants.city = df_restaurants.city.str.upper()
df_restaurants.state = df_restaurants.state.str.upper()

AttributeError: Can only use .str accessor with string values!

In [22]:
df_restaurants.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,food_type_one
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,NaN,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",None
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,NaN,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...",Burgers
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,NaN,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",None,American
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,NaN,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '...",Burgers
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,NaN,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...",Vietnamese


In [13]:
state_city_dict = df_restaurants.groupby('state')['city'].apply(set).to_dict()
state_city_dict = {state: list(cities) for state, cities in state_city_dict.items()}

# deleting XMS as its Merseyside zip code
state_city_dict.pop('XMS',None)
state_city_dict.keys()

AttributeError: 'str' object has no attribute 'str'

In [8]:
import json

with open('state_city_dict.json', 'w') as json_file:
    json.dump(state_city_dict, json_file, indent=4)


In [9]:
 state_names = {
    'AB': 'Alberta',          # 'AB' is not a standard US state abbreviation, might refer to Alberta, Canada
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'DE': 'Delaware',
    'FL': 'Florida',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'LA': 'Louisiana',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'NJ': 'New Jersey',
    'NV': 'Nevada',
    'PA': 'Pennsylvania',
    'TN': 'Tennessee',
}

# for key in list(state_city_dict.keys()):
#     if key in state_names.keys():
#         state_city_dict[state_names[key]] = state_city_dict.values()


full_state_city_dict = dict((state_names[key],value) for (key,value) in state_city_dict.items())

In [12]:
full_state_city_dict

{'Alberta': ['East Edmonton',
  'Downtown',
  'Eaux Claires',
  'St. Albert',
  'NW Edmonton',
  'West Edmonton',
  'Sherwood Park',
  'Beaumont',
  'St Albert',
  'EdMonton',
  'Edmonton',
  'Sherwood',
  'Enoch',
  'Saint Albert',
  'Southeast Edmonton'],
 'Arizona': ['TUCSON',
  'Casas Adobes',
  'South Tucson',
  'Picture Rocks',
  'Sahuarita',
  'Corona de Tucson',
  'Corona De Tucson',
  'Tuscon',
  'Vail',
  'Valencia West',
  'tucson',
  'Tucson',
  'Catalina',
  'Oro Valley',
  'Drexel Heights',
  'Marana',
  'Green Valley',
  'Mount Lemmon'],
 'California': ['Santa Barbara',
  'Santa  Barbara',
  'Carpinteria',
  'Summerland',
  'Montecito',
  'Goleta',
  'Truckee',
  'Isla Vista'],
 'Colorado': ['Vail'],
 'Delaware': ['Claymont',
  'Trolley Square',
  'New Castle',
  'Elsmere',
  'Greenville',
  'Newport',
  'Wilmington Manor',
  'wilmington',
  'Pike Creek',
  'Wilmington ',
  'Marshallton',
  'Stanton',
  'Aston',
  'Talleyville',
  'Wilmington',
  'Montchanin',
  'Christi